In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
class Model:
    
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            # input place holder
            self.X = tf.placeholder(tf.float32, [None,784])
            
            self.keep_prob = tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            # img 28 x 28 x 1(black/white)
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            
            # 0~9
            self.Y = tf.placeholder(tf.float32, [None,10])
            
            # L1 Img In Shape = (?,28,28,1)
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2, padding='SAME')
            dropout1 = tf.layers.dropout(inputs=pool1, rate=self.keep_prob, training=self.training)

            # L2 Img In Shape = (?,14,14,1)
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2, padding='SAME')
            dropout2 = tf.layers.dropout(inputs=pool2, rate=self.keep_prob, training=self.training)

            # L3 Img In Shape = (?,7,7,1)
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], strides=2,padding='SAME')
            dropout3 = tf.layers.dropout(inputs=pool3, rate=self.keep_prob, training=self.training)

            layer = tf.reshape(dropout3, [-1,128*4*4])
            
            # 
            W4 = tf.get_variable("W4",shape=[128*4*4,625],initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(layer,W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)

            W5 = tf.get_variable("W5",shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.hypothesis = tf.matmul(L4,W5)+b5

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.hypothesis,labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.0015).minimize(self.cost)

        prediction = tf.equal(tf.argmax(self.hypothesis,1),tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
            
            
    def prediction(self,x_test,keep_prob=1.0):
        return self.sess.run(self.hypothesis,feed_dict={self.X:x_test, self.keep_prob:keep_prob, self.training:False})

    def get_accuracy(self,x_test, y_test, keep_prob=1.0):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test,self.keep_prob:keep_prob, self.training:False})

    def train(self, x_train, y_train, keep_prob=0.7):
        return self.sess.run([self.cost,self.optimizer], feed_dict={self.X:x_train, self.Y:y_train, self.keep_prob:keep_prob, self.training:True})


In [14]:
tf.reset_default_graph()

training_epochs = 15
batch_size = 100

models = []
num_models = 6

sess = tf.Session()

for m in range(num_models):
    models.append(Model(sess,"model"+str(m)))

#init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess.run(tf.global_variables_initializer())


In [15]:
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs,batch_ys)
            avg_cost_list[m_idx] += c/total_batch

    print('Epoch : ', '%04d' % (epoch+1), 'cost =', avg_cost_list)

Epoch :  0001 cost = [ 0.62597504  0.63645738  0.67276016  0.6651895   0.64643619  0.64424733]
Epoch :  0002 cost = [ 0.25447358  0.24919691  0.24522438  0.26055225  0.27294251  0.25554156]
Epoch :  0003 cost = [ 0.21043509  0.20556418  0.20636098  0.21365405  0.22775251  0.21130497]
Epoch :  0004 cost = [ 0.1883496   0.18466272  0.18002822  0.19603202  0.19880734  0.19014246]
Epoch :  0005 cost = [ 0.18009201  0.17493964  0.17251407  0.17943334  0.18478007  0.17655075]
Epoch :  0006 cost = [ 0.16831243  0.16778671  0.1649102   0.17794784  0.17605095  0.16950436]
Epoch :  0007 cost = [ 0.16267668  0.16715226  0.16343248  0.16245999  0.1670017   0.16757156]
Epoch :  0008 cost = [ 0.16099456  0.1606795   0.15869848  0.16144221  0.1662918   0.16394776]
Epoch :  0009 cost = [ 0.16141117  0.15685447  0.15281362  0.16387178  0.16094825  0.15769541]
Epoch :  0010 cost = [ 0.15461947  0.1579871   0.14754788  0.1566767   0.16158759  0.15369106]
Epoch :  0011 cost = [ 0.14857391  0.15467592  0.1

In [16]:
labels = len(mnist.test.labels) #1000
prediction = np.zeros(labels*10).reshape(labels,10)

In [17]:
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy : ',m.get_accuracy(mnist.test.images,mnist.test.labels))
    p = m.prediction(mnist.test.images)
    prediction += p

ensemble_correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(mnist.test.labels,1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction,tf.float32))
print("ensemble accuracy : ", sess.run(ensemble_accuracy))

0 Accuracy :  0.9836
1 Accuracy :  0.983
2 Accuracy :  0.9881
3 Accuracy :  0.9848
4 Accuracy :  0.9853
5 Accuracy :  0.9874
ensemble accuracy :  0.9887
